In [6]:
import sys
sys.path.append('/home/av555/iclr/')

In [7]:
from architectures import *
from trainer import *
from loaders import *
from models import *
from utils import *

In [8]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from matplotlib import pyplot as plt

In [9]:
import cmocean
import cmocean.cm as cmo
import xarray as xr

ModuleNotFoundError: No module named 'xarray'

In [ ]:
!pip3 install cmocean

## Utils

In [4]:
def load_model(path, model):

    checkpoint = torch.load(path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    model.eval()
    return model, epoch, loss


In [5]:
def eval_model(model, val_loader):
    preds = []
    targets = []

    with torch.no_grad():
        for task in val_loader:
            task["dists"] = task["dists"][0,...]
            out = model(task)
            preds.append(out)
            targets.append(task["y_target"])

    #concat full arrays
    preds = torch.cat(preds, dim = 0)
    targets = torch.cat(targets, dim = 0)

    #To cpu
    preds = preds.cpu().numpy()
    targets = targets.cpu().numpy()

    return preds, targets

In [6]:
def get_val_stats_bs(predictions, targets):
        

        maes = np.zeros(predictions.shape[1])
        spearmans = np.zeros(predictions.shape[1])
        pearsons = np.zeros(predictions.shape[1])

        for st in range(predictions.shape[1]):
            # Get station
            true_mean = targets[:, st]
            pred_mean = predictions[:, st,0]
            # Remove nans
            pred_mean = pred_mean[~np.isnan(true_mean)]
            true_mean = true_mean[~np.isnan(true_mean)]

            try:
                maes[st] = np.mean(np.abs(true_mean - pred_mean))
                pearsons[st] = scipy.stats.pearsonr(pred_mean, true_mean)[0]
                spearmans[st] = scipy.stats.spearmanr(pred_mean, true_mean).correlation
            except:
                maes[st] = np.nan
                pearsons[st] = np.nan
                spearmans[st] = np.nan
                continue
        print("Mean absolute error: {}".format(np.nanmedian(maes)))
        print("Pearson correlation: {}".format(np.nanmedian(pearsons)))
        print("Spearman correlation: {}".format(np.nanmedian(spearmans)))

In [7]:
use_cpu = True
device = torch.device('cpu') if use_cpu else torch.device('cuda')


## Data loader

In [23]:
class UKHighResLoader(Dataset):
  
  def __init__(self, 
               data_dir, 
               device):

        super().__init__()

        self.DATA_DIR = data_dir
        self.device=device

        self.x_context = np.load(
            self.DATA_DIR+"context/x_context_coarse_final.npy")
        self.y_context = np.load(
            self.DATA_DIR+"context/y_context_coarse_final.npy", mmap_mode="r")[:8766,...]
        
        self.x_target = np.load(
            self.DATA_DIR+"spatial_samples/target_uk.npy")
        self.elev = np.load(
            self.DATA_DIR+"spatial_samples/elev_uk.npy")

        # Calculate dists and scale
        self.dists = get_dists(self.x_target, 
                                self.x_context[...,0], 
                                self.x_context[...,1])
        #x_scaler = MinMaxScaler().fit(self.dists.reshape(-1,1))
        #self.dists = x_scaler.transform(self.dists.reshape(-1,1)).reshape(self.dists.shape)

  def __len__(self):
        'Denotes the total number of samples'
        return self.y_context.shape[0]

  def __getitem__(self, index):
        'Generates one sample of data'
        y_context = torch.from_numpy(self.y_context[index,...]).float()
        dists = torch.from_numpy(self.dists).float()
        elev = torch.from_numpy(self.elev).float()
        x_target = torch.from_numpy(self.x_target).float()

        return {"y_context":y_context.to(self.device),  
                "x_target":x_target.to(self.device),
                "dists":dists.to(self.device),
                "elev":elev.to(self.device)}

## Load model and plot

In [38]:
use_cpu = True
device = torch.device('cpu') if use_cpu else torch.device('cuda')

In [39]:
# data
val_ds = UKHighResLoader("/home/av555/iclr/data/", device=device)
val_loader = DataLoader(
    val_ds, batch_size = 16, shuffle = False)

In [40]:
# ConvCNP

convcnp_path = "/home/av555/iclr/exps/convCNP_value_all/epoch_459"
concnp_model = convCNP(
    in_channels=25)

concnp_model, convcnp_epoch, convcnp_loss = load_model(convcnp_path, concnp_model)

In [41]:
for i in val_loader:
    test = i
    break

In [42]:
for key in test.keys():
    print(key)
    print(test[key].shape)

y_context
torch.Size([16, 25, 32, 19])
x_target
torch.Size([16, 2838, 2])
dists
torch.Size([16, 2838, 32, 19])
elev
torch.Size([16, 2838, 3])


In [3]:
test["dists"] = test["dists"][0,...]
out = concnp_model(test)

NameError: name 'test' is not defined

In [4]:
x = val_ds.x_target[...,0].reshape(66,43)
y = val_ds.x_target[...,1].reshape(66,43)
z = out[0,:,0].detach().numpy().reshape(66,43)

NameError: name 'val_ds' is not defined

In [5]:
p = plt.pcolormesh(x, y, z, cmap="seismic", vmin=-25, vmax=25) #, transform=rotated_pole)
c = plt.colorbar(p, orientation="horizontal", pad=0.01, shrink=0.8)

NameError: name 'x' is not defined